#Song Lyrics Generator
For this project, I want to generate lyrics using data science. I will generate lyrics by training a Markov Chain Model by feeding it lyrics so that I can generate a song in a style similar to the lyrics that I inputted.

#Step 1: Web Scraping Song Lyrics
For fun and out of my own morbid curiosity, I'm going to be using the lyrics from my five least favorite Logic songs, Icy, clickbait, Bohemian Trapsody, Everyday, and Take It Back. I will be using the lyrics from these songs to recreate some equally horrible songs myself.

In [0]:
import requests
import time

from bs4 import BeautifulSoup

In [0]:
website = 'https://www.metrolyrics.com/'
end = '-lyrics-logic.html'
lyrics = []

In [0]:
def find_lyrics(song_name):
  response = requests.get(website + song_name + end)
  soup = BeautifulSoup(response.content, "html.parser")
  ps = soup.find_all('p')
  return ps

In [0]:
songs = ['icy', 'clickbait', 'bohemian-trapsody', 'everyday', 'take-it-back']
pss = []
for song in songs:
  pss.append(find_lyrics(song))
  time.sleep(0.5)

In [0]:
icy_lyrics = ''
for pnum in range(1, len(pss[0]) - 1):
  if pnum == 5:
    continue #ignore Gucci Mane guest verse
  lyric_list = pss[0][pnum].text.split('\n')[1:]
  lyric_chunk = ' \n '.join(lyric_list)
  icy_lyrics += ' \n ' + lyric_chunk
lyrics.append(icy_lyrics[3:])

In [0]:
clickbait_lyrics = ''
for pnum in range(1, len(pss[1]) - 1):
  lyric_list = pss[1][pnum].text.split('\n')[1:]
  lyric_chunk = ' \n '.join(lyric_list)
  clickbait_lyrics += ' \n ' + lyric_chunk
lyrics.append(clickbait_lyrics[3:])

In [0]:
bohemian_lyrics = ''
for pnum in range(1, len(pss[2]) - 1):
  lyric_list = pss[2][pnum].text.split('\n')
  lyric_chunk = ' \n '.join(lyric_list)
  bohemian_lyrics += ' \n ' + lyric_chunk
lyrics.append(bohemian_lyrics[3:])

In [0]:
everyday_lyrics = ''
for pnum in range(1, len(pss[3]) - 1):
  lyric_list = pss[3][pnum].text.split('\n')[1:]
  lyric_chunk = ' \n '.join(lyric_list)
  everyday_lyrics += ' \n ' + lyric_chunk
lyrics.append(everyday_lyrics[3:])

In [0]:
tib_lyrics = ''
for pnum in range(2, len(pss[4]) - 1):
  lyric_list = pss[4][pnum].text.split('\n')
  lyric_chunk = ' \n '.join(lyric_list)
  tib_lyrics += ' \n ' + lyric_chunk
lyrics.append(tib_lyrics[3:])

In [10]:
#The lyrics to Icy
print(lyrics[0])

I'm icy, I'm clean 
 I'm icy 
 I'm icy (I'm icy, I'm clean) 
 I'm clean (I'm icy, I'm clean) 
 I'm a bad motherfucker (I'm icy, I'm clean) 
 I'm mean (I'm icy) 
 I'm icy (I'm icy, I'm clean) 
 I'm clean (I'm icy, I'm clean) 
 I'm a bad motherfucker (I'm icy, I'm clean) 
 I'm mean (I'm icy, I'm clean) 
 I had a plot as a kid to get rich 
 Hah, listen, lil' bitch 
 Got the game in my hand like a Switch 
 It's just me and my clique 
 Rap and get money, rap and get money 
 Said rap and get money, motherfucker 
 Ain't this shit funny? 
 Don't do it for money, don't do it for money 
 But if it come down to the money 
 I got a lot 
 I got a lot, like a lot 
 More than I need 
 I got children to feed, I got places to be 
 I got bitches to fuck, let me see your ID 
 Nah, I'm just playin', I'm playin' 
 You know that I'm playin', don't know what I'm sayin', no 
 Know that we chillin', we vibin' 
 We smokin' and drinkin', we gettin' this money though 
 Follow me, follow me, follow me 
 All the wa

#Step 2: The Bigram Markov Chain Model
In a dictionary, I am storing every pair of consecutive words (every birgram) as a tuple in all of the five songs and mapping each pair to the list of "next" words.

For example, for the line `"I'm Icy, I'm Clean"`

The pair of words `("I'm", "Icy")` is mapped to the word following the pair,`["I'm"]`. Likewise, the pair of words `("Icy, "I'm")` is mapped to `["Clean"]`.

Besides words, I am also storing starting points (`"<START>"`), ending points (`"<END>"`), and line breaks (`"<N>"`) to capture how songs and lines tend to start and end.

In [0]:
import re
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dictionary that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """
    chain = {(None, "<START>"): []}
    for lyric in lyrics:
        words = re.sub('\n', '<N>', re.sub(r'[^\w\s]', '', lyric)).split(' ')
        #normalize text lyrics by getting rid of puncuation

        chain[(None, "<START>")].append(words[0])
        chain[("<START>", words[0])] = [words[1]]

        for i in range(len(words) - 2):
          word = words[i]
          nextword = words[i + 1]
          nextnext = words[i + 2]
          if (word, nextword) not in chain:
            chain[(word, nextword)] = []
          if (nextword, nextnext) not in chain:
            chain[(nextword, nextnext)] = []
          chain[(word, nextword)].append(nextnext)

        chain[(words[len(words) - 2], words[len(words) - 1])].append('<END>')

    return chain

We can take a peek into our model by seeing how songs tend to start.

In [12]:
chain = train_markov_chain(lyrics)

print(chain[(None, "<START>")])

['Im', 'You', 'Every', 'Yeah', 'Take']


#Step 3: Generate Lyrics!
I will be randomly generating lyrics using the Bigram Markov Chain I created above. I will begin at the `(None, "<START>")` bigram and I will randomly choose a word from the corresponding list of words to be the first word in the song. At each step, I will randomly sample the next word from the list of words that follow the current bigram (i.e. the last two randomly generated words). I will continue to randomly generate lyrics until I sample `"<END>"`. And this should give us a complete song!

In [0]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """ 
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain[(None, "<START>")]))
    prev = words[0]
    current = random.choice(chain[('<START>', prev)])
    
    while current != '<END>':
      words.append(current)
      new = random.choice(chain[(prev, current)])
      prev = current
      current = new
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [15]:
print(generate_new_lyrics(chain))

You are amazing you are amazing you are amazing you are amazing you are amazing you are amazing you are amazing you are 
 You are amazing you are amazing you are amazing you are forgettable you are forgettable you are amazing you are amazing you are for 
 Elevated feelin elevated took a hit of weed and shit escalated 
 Feeling innovative Ima integrate it yeah thats how Ima buck back 
 Take it back 
 Take it way back to about 1990 
 Shady Grove Hospital in Rockville Maryland 
 7 36 am where a biracial baby was born equal 
 We didnt live with you we didnt live with you 
 Promise that Id risk it all for you 
 Promise that Id risk it all for you 
 Promise to reveal my all for you 
 And when I break down my wall for you 
 Thats when Ill break down my wall for you 
 Thats the shit that happened to this kid 
 And when I break down my wall for you 
 Promise to reveal my all for you 
 Promise to reveal my all for you 
 Oh na na 
 Nananana Hey 
 Oh no no I cant believe 
 As long as theyre not hu

#Conclusion
It's a success! This song sucks! The generated song is a beautiful mess of the combined lyrics of the five inputted songs. Could not be happier to have birthed something so atrocius.